# Project Description
School choice in renting vs. buying. The choice between renting versus buying housing has been a highly debated topic in therecent years, especially with high volatility of the prices of real estate.  Zillow the mostused real estate search platform in the US which offers information on activity on realestate and rental listings (in terms of the numbers of views) as well as detailed informationon the area where the property is located including “GreatSchools” rating.  The projectis devoted to evaluate if the quality of schools has an effect of the decision of individualsto buys vs rent housing based on the relative activity on rental and for sale listings. The following are the specific steps this project would take:
1.  Identify a market which is going to be the focus of your analysis.  This could be acity, a county or a specific zip code.  Provide motivation for your choice.
2.  Provide an approach to collect the relevant data from Zillow on the current rentaland  for  sale  properties  in  your  market  as  well  as  specific  information  on  thoseproperties  including  prices,  property  characteristics  (square  footage,  number  of bedrooms) as well as the information on the school quality and the number of userviews of the property.  Collect the dataset of sufficient size (you determine whichsize is sufficient for your analysis).
3.  Provide an approach to make the prices of rental and for sale listings compatible.Note  that  rental  prices  are  per  month  while  the  for  sale  prices  are  for  the  entireproperty purchase.
4.  Using the number of user views as a proxy for the relative interest of Zillow users in a specific listing, construct a model that would identify how users select to viewrental listings versus for sale listings and the role the quality of schools plays in thischoice

# High-Level Overview
1. Iterate through main Zillow listing pages to collect links to individual properties.
2. Iterate through individual house links, clicking in to load school quality data. Scrape both 'for sale' and 'rental' listings. Workflow adapted from
https://www.scrapingdog.com/blog/scrape-zillow/ , with substantial adjustments.
3. Impute predicted sale price for rental properties from monthly rental cost.
4. Apply generalized linear model to scraped data to explore factors affecting home price. We evaluate the hypothesis that nearby school quality positively correlates with price for homes for sale but not rentals.

# Main Technical Challenges
1. Zillow applies several anti-scraper techniques, including selenium detection, CAPTCHA, and IP blocking.
  1. After setting a mock header and enabling JS rendering, our scraper escaped detection but began hitting an unpassable CAPTCHA after multiple webpage accesses. We need to access all 800 listings individually to load school quality data, so this was not workable.
  2. We solved this issue by dynamically loading a fresh IP for each new pageview using a proxy API.
2. Even after avoiding CAPTCHA, our scraper would be sporadically blocked from capturing the HTML data about half of the time.
  1. Since this issue occured randomly and did not stably fail for any given link, we simply iterated the scraper over the remaining "bad" links multiple times to collect the complete dataset.




In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install selenium-wire
!pip install pandas
!pip install requests
!pip install httpx parsel loguru
!pip install geopy

# *Webscraper*
Capture the listings on all 20 pages displayed for Austin, TX by writing the full HTML to a file.

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import math

response = requests.get(
  url='https://proxy.scrapeops.io/v1/',
  params={
      'api_key': 'API-KEY', 
      'url': 'https://www.zillow.com/glen-allen-va/1_p', ## PerimeterX protected website
      'bypass': 'perimeterx'
  }
)

with open(r'dataPages.txt', 'w', encoding="utf-8") as fp:
  fp.write("%s\n $$$nextHTML$$$\n" % response.text)

# compute number of pages
  soup = BeautifulSoup(response.text,'html.parser')
  num_results = soup.find("span", {"class":"result-count"}).text
  num_results = num_results.replace(' results', '')
  num_results = int(num_results.replace(',', ''))
  num_pages = int(math.ceil(num_results/40))
  print(f'Total Number of Results: {num_results}')
  print(f'Total Number of Pages: {num_pages}')

  for i in range(2,num_pages+1):
    rep = requests.get(
        url='https://proxy.scrapeops.io/v1/',
        params={
          'api_key': 'API-KEY',
          'url': 'https://www.zillow.com/glen-allen-va/'+str(i)+'_p/',
          'bypass': 'perimeterx'
        }
    )
    fp.write("%s\n $$$nextHTML$$$\n" % rep.text)

Then, extract the links to each individual property while filtering duplicates.

In [ ]:
from bs4 import BeautifulSoup
import re

links = set()

with open('dataPages.txt', 'r') as f: #Change for file location
    content = f.read()

    soup = BeautifulSoup(content, 'html.parser')

    for a in (soup.find_all("a", href=re.compile("_zpid"))):
        links.add(a["href"])

links = list(links)
print(links)

# *Data Extraction*

Finally, loop over all individual properties and scrape:

*   Score of 3 nearest schools
*   Page views
*   Page saves (analogous to liking)
*   Time on market
*   Address
*   Square footage
*   Price

We then join these data in a dataframe and export to CSV for downstream analysis. This separation allowed us to work on data harvesting and the analysis/visualization in parallel using a small subset of the data before the scraper was finalized.


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

def processHouse(houseFile, df):
    obj = {}
    with open(houseFile, 'r', encoding='utf-8') as fp:
        soup = BeautifulSoup(fp, 'html.parser')

        try:
            #School score list ordered from ES, MS, HS
            obj['schoolscore'] = [str(soup.find_all("span", class_="Text-c11n-8-84-0__sc-aiai24-0 gTVYcr")[-3])[51:-7], str(soup.find_all("span", class_="Text-c11n-8-84-0__sc-aiai24-0 gTVYcr")[-2])[51:-7], str(soup.find_all("span", class_="Text-c11n-8-84-0__sc-aiai24-0 gTVYcr")[-1])[51:-7]]
        except:
            obj['schoolscore'] = None
        try:
            obj['views'] = str(soup.find_all("dt")[1])[12:-14]
        except:
            obj['views'] = None
        try:
            obj['saves'] = str(soup.find_all("dt")[2])[12:-14]
        except:
            obj['saves'] = None
        try:
            obj['timeonmarket'] = str(soup.find_all("dt")[0])[12:-14]
        except:
            obj['timeonmarket'] = None
        try:
            obj['addr'] = str(soup.find("title"))[7:-32]
        except:
            obj['addr'] = None
        try:
            index = str(soup.find_all("meta")).find("$") + 1
            space = str(soup.find_all("meta"))
            price = ""
            while True:
              if space[index] != " " and space[index] != ".":
                price += str(space[index])
                index += 1
              else:
                break
            price = int(price.replace(",", ""))
            obj['price'] = price
        except:
            obj['price'] = None
        try:
          index = int(str(soup.find_all("meta")).find("Square Feet")) - 2
          solset = str(soup.find_all("meta"))

          if index > -1:
              comma_counter = 0
              sqft = ""
              while True:
                  if solset[index] != "," and solset[index] != " ":
                      sqft += str(solset[index])
                      index -= 1
                  else:
                      if solset[index] == ",":
                          if comma_counter < 1:
                              comma_counter += 1
                              index -= 1
                          else:
                              break
                      else:
                          break

              obj['sqft'] = print(sqft[::-1])

          else:
              index = int(str(soup.find_all("meta")).find("sq. ft.")) - 2
              if index > -1:
                  solset = str(soup.find_all("meta"))
                  comma_counter = 0
                  sqft = ""
                  while True:
                      solset[index]
                      if solset[index] != "," and solset[index] != " ":
                          sqft += str(solset[index])
                          index -= 1
                      else:
                          if solset[index] == ",":
                              if comma_counter < 1:
                                  comma_counter += 1
                                  index -= 1
                              else:
                                  break
                          else:
                              break

                  obj['sqft'] = print(sqft[::-1])

              else:
                  index = int(str(soup.find_all("meta")).find("Acres")) - 2
                  solset = str(soup.find_all("meta"))
                  period_counter = 0
                  sqft = ""
                  while True:
                      solset[index]
                      if solset[index] != "." and solset[index] != " ":
                          sqft += str(solset[index])
                          index -= 1
                      else:
                          if solset[index] == ".":
                              if period_counter < 1:
                                  sqft += solset[index]
                                  period_counter += 1
                                  index -= 1
                              else:
                                  break
                          else:
                              break

                  obj['sqft'] = print(int(float(sqft[::-1])*43560))
        except:
            obj['sqft'] = None
        obj['rentorbuy'] = 'buy'
        df.loc[len(df)] = obj

In [ ]:
import os
import linecache
from tqdm import tqdm
import pandas as pd

df = pd.DataFrame(data=[], columns=['price', 'rentorbuy', 'sqft','price_per_sqft', 'schoolscore', 'views', 'saves', 'timeonmarket', 'lat', 'long', 'addr'])

for filename in tqdm(os.listdir(r'FILEPATH')):
    f = os.path.join(r'FILEPATH', filename)
    if os.path.isfile(f):
        processHouse(f, df)

In [ ]:
# Lambdas for data cleaning
def inter(x):
    try:
        y = int(x)
        return y
    except:
        return None
def inter2(x):
    if isinstance(x, str):
        return int(x.replace(",",""))
    else:
        return x
def inter3(x):
    if x[-4:] == "days":
        return int(x.replace(' days',''))
    else:
        return int(x.replace(' day',''))

In [ ]:
df['sqft'] = df['sqft'].apply(inter)
df['price_per_sqft'] = df['price'].div(df['sqft'])
# Filter out data with missing elements
df_good = df.loc[df['schoolscore'].notna()].loc[df['sqft'].notna()]

In [ ]:
# Drop rows with missing data for schoolscore
for i, row in df_good.iterrows():
    if row['schoolscore'][0] not in [0,1,2,3,4,5,6,7,8,9,10,'0','1','2','3','4','5','6','7','8','9','10']:
        df_good.drop(i, inplace=True)

# Then turn remaining schoolscore entries to ints
for i, row in df_good.iterrows():
    for j in range(len(row['schoolscore'])):
        row['schoolscore'][j] = int(row['schoolscore'][j])

df_good['views'] = df_good['views'].apply(inter2)
df_good['saves'] = df_good['saves'].apply(inter2)
df_good['timeonmarket'] = df_good['timeonmarket'].apply(inter3)

In [ ]:
df_good[20:40] # Subset of scraped 'for sale' property data

,price,rentorbuy,sqft,price_per_sqft,schoolscore,views,saves,timeonmarket,lat,long,addr
26,715000,buy,1669.0,428.400240,"[7, 5, 7]",644,55,2,0,0,"12305 Black Angus Dr, Austin, TX 78727"
27,350000,buy,792.0,441.919192,"[5, 2, 4]",1932,20,61,0,0,"3209 E 16th St, Austin, TX 78721"
28,1299500,buy,3770.0,344.694960,"[5, 5, 7]",2028,54,59,0,0,"5204 Pony Chase, Austin, TX 78727"
29,1350000,buy,2376.0,568.181818,"[9, 5, 6]",1297,64,16,0,0,"2504 Hartford Rd, Austin, TX 78703"
32,875000,buy,2278.0,384.108867,"[8, 8, 8]",505,29,3,0,0,"10816 Redmond Rd, Austin, TX 78739"
33,639000,buy,852.0,750.000000,"[6, 5, 6]",183,7,2,0,0,"360 Nueces St APT 1407, Austin, TX 78701"
34,415000,buy,2199.0,188.722146,"[6, 4, 3]",409,32,3,0,0,"5601 Apple Orchard Ln, Austin, TX 78744"
35,650000,buy,2554.0,254.502741,"[5, 5, 4]",425,24,2,0,0,"10621 Beard Ave, Austin, TX 78748"
36,7800000,buy,6374.0,1223.721368,"[7, 6, 7]",1589,52,9,0,0,"3407 Monte Vista Dr, Austin, TX 78731"
37,290000,buy,1440.0,201.388889,"[6, 8, 7]",312,17,2,0,0,"2123 Tishomingo Trl, Austin, TX 78734"


In [ ]:
df_good.to_csv(r'C:\Users\James\AlgoEcon\houses.csv')

We then repeated the same process to scrape rental data and saved it to a separate CSV. The duplicated analysis has been left out for brevity.

This code imports latitude and longitude data for each house given the address using Google's geocoding API. These data are used later for visualization purposes.

In [ ]:
import requests
import codecs
from pathlib import Path
GOOGLE_API_KEY = Path('AlgoEcon/geocodingAPIKEY.txt').read_text()

def extract_lat_long_via_address(address_or_zipcode):
    lat, lng = None, None
    api_key = GOOGLE_API_KEY
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={address_or_zipcode}&key={api_key}"
    r = requests.get(endpoint)
    if r.status_code not in range(200, 299):
        return None, None
    try:
        '''
        This try block incase any of our inputs are invalid. This is done instead
        of actually writing out handlers for all kinds of responses.
        '''
        results = r.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
    except:
        pass
    return lat, lng

In [ ]:
import pandas as pd

houses = pd.read_csv("AlgoEcon/houses.csv")
rentals = pd.read_csv("AlgoEcon/rentals.csv")

coords = houses['addr'].apply(extract_lat_long_via_address)
houses['lat'] = [row[0] for row in coords]
houses["long"] = [row[1] for row in coords]

coords = rentals['addr'].apply(extract_lat_long_via_address)
rentals['lat'] = [row[0] for row in coords]
rentals["long"] = [row[1] for row in coords]

houses.to_csv("AlgoEcon/housesLATLNG.csv")
rentals.to_csv("AlgoEcon/rentalsLATLNG.csv")

Here we loop back over "bad" links where the scraper failed and try again to capture them before rerunning.

In [ ]:
links = set()

with open('alllinks.txt', 'r') as fp:
    lines = fp.readlines()
    for line in lines:
        links.add(line)
links = list(links)

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import math
import linecache

bad_links = []

for i in range(0, 159):
    with open('house' + str(i) + '.txt', 'r') as f:
            content = f.readlines()

            if len(content) != 0:
                if content[0][2:8] == "status":
                    bad_links.append(i)
                else:
                    if (content[1][:7] == "<title>"):
                        continue
                    else:
                        bad_links.append(i)

print(bad_links)

for i in range(len(bad_links)):
    rep = requests.get(
                url='https://proxy.scrapeops.io/v1/',
                params={
                    'api_key': '<API-KEY>',
                    'url': links[bad_links[i]],
                    'bypass': 'perimeterx',
                    'render_js': 'true',
                    'optimize_request':'true',
                    'residential': 'true',
                    'country': 'us',
                }
        )

    with open(r'data_new\house' + str(i) +'.txt', 'w', encoding="utf-8") as fp:
       fp.write("%s" % rep.text)



The main datasets have now been prepared. Analysis and visualization proceeds in the provided R code.